# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from train_raintomorrow import clean_data
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute

from azureml.train.automl import AutoMLConfig
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.core import Pipeline


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
This AutoML experiment will be performed in order to investigate of *rain tomorrow* dataset. This dataset has xx columns and ... rows....

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-rain'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


Workspace name: coursera_paes
Azure region: brazilsouth
Subscription id: 90b94744-e13a-4ecb-a955-1cbb85261a5d
Resource group: defaultresourcegroup-brazilsouth


# Create a Cluster

In [3]:

#cluster_name = "GIVE_A_CLUSTER_NAME"
cluster_name = "vzeizer1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

try:
    cpu_cluster=ComputeTarget(workspace=ws,name=cluster_name)
    print('Cluster already exists!')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",max_nodes=4)
    cpu_cluster = ComputeTarget(ws,cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Cluster already exists!

Running


In [4]:
#path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
path='https://raw.githubusercontent.com/vzeizer/Udacity_Capstone_MLEngAzure/master/weatherAUS.csv'
data_rain=TabularDatasetFactory.from_delimited_files(path)

if "training" not in os.listdir():
    os.mkdir("./training")

# I won't clean the data because I will enable featurization

#x, y = clean_data(data_rain)### YOUR DATA OBJECT HERE ###

# split into train and test data
#x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=42)

#data_train=pd.concat([x_train,y_train],axis=1)
#data_test=pd.concat([x_test,y_test],axis=1)
#data_train=x_train.copy()
#data_test=x_test.copy()
#data_train['y']=y_train
#data_test['y']=y_test
#ds=data_train.to_csv('training/train.csv')
#ds=ws.get_default_datastore()
#ds.upload(src_dir='training',target_path='training',overwrite=True)
#ds_train=Dataset.Tabular.from_delimited_files(path=[(ds,'training/train.csv')])

df_rain = data_rain.to_pandas_dataframe()
df_rain.describe()

,MinTemp,MaxTemp,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142806.000000,140953.000000,130395.00000,130432.000000,143693.000000,141851.00000
mean,12.194034,23.221348,68.880831,51.539116,1017.64994,1015.255889,16.990631,21.68339
std,6.398495,7.119049,19.029164,20.795902,7.10653,7.037414,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,980.50000,977.100000,-7.200000,-5.40000
25%,7.600000,17.900000,57.000000,37.000000,1012.90000,1010.400000,12.300000,16.60000
50%,12.000000,22.600000,70.000000,52.000000,1017.60000,1015.200000,16.700000,21.10000
75%,16.900000,28.200000,83.000000,66.000000,1022.40000,1020.000000,21.600000,26.40000
max,33.900000,48.100000,100.000000,100.000000,1041.00000,1039.600000,40.200000,46.70000


In [5]:
df_rain.head(3)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NA,NA,W,44,W,...,71.0,22.0,1007.7,1007.1,8,NA,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0,NA,NA,WNW,44,NNW,...,44.0,25.0,1010.6,1007.8,NA,NA,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0,NA,NA,WSW,46,W,...,38.0,30.0,1007.6,1008.7,NA,2,21.0,23.2,No,No


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
# let us run the automl for 20 minutes
# 
automl_settings = {"experiment_timeout_minutes":30,
    "task":'classification',
    "primary_metric":'accuracy',
    "compute_target":cpu_cluster,
    "featurization":'auto',
    "training_data":data_rain,
#    "label_column_name":'y',
    "label_column_name":"RainTomorrow",
    "n_cross_validations":5,
    "enable_early_stopping":True}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)


#ds = ws.get_default_datastore()
#metrics_output_name = 'metrics_output'
#best_model_output_name = 'best_model_output'

#metrics_data = PipelineData(name='metrics_data',
#                           datastore=ds,
#                           pipeline_output_name=metrics_output_name,
#                           training_output=TrainingOutput(type='Metrics'))
#model_data = PipelineData(name='model_data',
#                           datastore=ds,
#                           pipeline_output_name=best_model_output_name,
#                           training_output=TrainingOutput(type='Model'))
#automl_step = AutoMLStep(
#    name='automl_module',
#    automl_config=automl_config,
#    outputs=[metrics_data, model_data],
#    allow_reuse=True
#)

In [7]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)
automl_run.wait_for_completion(show_output=True)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-rain,AutoML_7634308b-8faa-4f34-9c19-53cef7e3654e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
automl-rain,AutoML_7634308b-8faa-4f34-9c19-53cef7e3654e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+-----

{'runId': 'AutoML_7634308b-8faa-4f34-9c19-53cef7e3654e',
 'target': 'vzeizer1',
 'status': 'Completed',
 'startTimeUtc': '2023-03-06T11:00:34.231958Z',
 'endTimeUtc': '2023-03-06T11:59:31.176563Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'vzeizer1',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-rain","subscription_id":"90b94744-e13a-4ecb-a955-1cbb85261a5d","resource_group":"defaultresourcegroup-brazilsouth","workspace_name":"coursera_paes","region":"brazilsouth","compute_target":"vzeizer1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_ru

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Retrieve and save your best automl model.
best_run,best_model_output = automl_run.get_output()
print('best model:',best_run.id,best_model_output)


best model: AutoML_7634308b-8faa-4f34-9c19-53cef7e3654e_21 Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 StackEnsembleClassifier(base_learners=[('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('xgboostclassifier', XGBoostClassifier(n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': '

In [65]:
#TODO: Save the best model
# I am not sure whether saving like that will work
#saved_model=best_run.register_model(model_name='bestmodel_all',model_path=pp)
from azureml.core.model import Model
saved_model = Model.register(ws, model_path='bestmodel_all.joblib', model_name='bestmodel_all')#, tags=best_run_metrics)

Registering model bestmodel_all


In [66]:
#pp

In [67]:
1

1

In [68]:
import joblib
#best_run.download_files(output_directory='./')
param_values = best_run.get_details()['runDefinition']['arguments']
#save the best model obtained
joblib.dump(best_model_output, filename='bestmodel_all.joblib')

['bestmodel_all.joblib']

In [69]:
best_run_metrics = best_run.get_metrics()

print('The best run automl model\n', best_run)

print('\nThe Id for best run is:', best_run.id)
print('The Accuracy: is', best_run_metrics['accuracy'])

The best run automl model
 Run(Experiment: automl-rain,
Id: AutoML_7634308b-8faa-4f34-9c19-53cef7e3654e_21,
Type: azureml.scriptrun,
Status: Completed)

The Id for best run is: AutoML_7634308b-8faa-4f34-9c19-53cef7e3654e_21
The Accuracy: is 0.8508318438058572


In [70]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/vzeizer1/code/Users/vzeizer


In [71]:
os.environ['AZUREML_MODEL_DIR'] = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/vzeizer1/code/Users/vzeizer"

In [72]:
from azureml.core.model import Model

#pp=Model.get_model_path('bestmodel_all.joblib')
pp=os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'bestmodel_all.joblib')
m1=joblib.load(pp)
#import pickle
#with open(pp, 'rb') as f:
#    m1 = pickle.load(f)


In [73]:
m1

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
                                                  StackEnsembleClassifier(base_learners=[('1', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('xgboostclassifier', XGBoostClassifier(n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_trainin

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [74]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.model import Model
#inference_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./score.py')



# from: "https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-deploy-and-where?tabs=python"

#env = Environment.get(workspace=ws, name="AzureML-AutoML")
from azureml.automl.core.shared import constants

best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'AutoML_env.yml') 
model_env = Environment.from_conda_specification(name="AutoML_env", file_path="AutoML_env.yml")
#model_automl=Model(ws,'bestmodel_automl')

service_name = 'bestmodel-service'

inference_config = InferenceConfig(entry_script='score.py', environment=model_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
#            ,enable_app_insights=True,auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[saved_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)



Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-03-06 12:44:57+00:00 Creating Container Registry if not exists.
2023-03-06 12:44:57+00:00 Registering the environment.
2023-03-06 12:44:59+00:00 Use the existing image.
2023-03-06 12:44:59+00:00 Generating deployment configuration.
2023-03-06 12:45:00+00:00 Submitting deployment to compute.
2023-03-06 12:45:08+00:00 Checking the status of deployment bestmodel-service..
2023-03-06 12:47:25+00:00 Checking the status of inference endpoint bestmodel-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [75]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-03-06T12:47:15,978091000+00:00 - iot-server/run 
2023-03-06T12:47:15,978091000+00:00 - rsyslog/run 
2023-03-06T12:47:15,988352800+00:00 - gunicorn/run 
2023-03-06T12:47:15,991823200+00:00 | gunicorn/run | 
2023-03-06T12:47:15,996741200+00:00 | gunicorn/run | ###############################################
2023-03-06T12:47:16,000215600+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-03-06T12:47:16,002616000

In [76]:
from azureml.core.webservice import LocalWebservice
port=9000
deployment_config = LocalWebservice.deploy_configuration(port=port)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [77]:

dict(df_rain.iloc[2])

{'Date': Timestamp('2008-12-03 00:00:00'),
 'Location': 'Albury',
 'MinTemp': 12.9,
 'MaxTemp': 25.7,
 'Rainfall': '0',
 'Evaporation': 'NA',
 'Sunshine': 'NA',
 'WindGustDir': 'WSW',
 'WindGustSpeed': '46',
 'WindDir9am': 'W',
 'WindDir3pm': 'WSW',
 'WindSpeed9am': '19',
 'WindSpeed3pm': '26',
 'Humidity9am': 38.0,
 'Humidity3pm': 30.0,
 'Pressure9am': 1007.6,
 'Pressure3pm': 1008.7,
 'Cloud9am': 'NA',
 'Cloud3pm': '2',
 'Temp9am': 21.0,
 'Temp3pm': 23.2,
 'RainToday': 'No',
 'RainTomorrow': 'No'}

In [78]:
df_rain['Date'].astype(str).iloc[2]

'2008-12-03'

In [79]:
#data = json.dumps({"data":df_rain[:5].drop(['y'], axis=1)})
#service.run(data)


In [80]:
Timestamp('2008-12-03 00:00:00').strftime('%Y%m%d%H%M%S')


'20081203000000'

In [81]:
service.token_auth_enabled

False

In [82]:
import requests
import json
from pandas import Timestamp
uri = service.scoring_uri
#requests.get("http://localhost:"+str(port))
headers = {"Content-Type": "application/json"}
data = {"data":
[{'Date': '2008-12-03',
 'Location': 'Albury',
 'MinTemp': 12.9,
 'MaxTemp': 25.7,
 'Rainfall': '0',
 'Evaporation': 10,
 'Sunshine': 2,
 'WindGustDir': 'WSW',
 'WindGustSpeed': '46',
 'WindDir9am': 'W',
 'WindDir3pm': 'WSW',
 'WindSpeed9am': '19',
 'WindSpeed3pm': '26',
 'Humidity9am': 38.0,
 'Humidity3pm': 30.0,
 'Pressure9am': 1007.6,
 'Pressure3pm': 1008.7,
 'Cloud9am': 31,
 'Cloud3pm': '2',
 'Temp9am': 21.0,
 'Temp3pm': 23.2,
 'RainToday': 'No'},{'Date': '2008-12-03',
 'Location': 'Albury',
 'MinTemp': 22.9,
 'MaxTemp': 45.7,
 'Rainfall': '0',
 'Evaporation': 20,
 'Sunshine': 10,
 'WindGustDir': 'WSW',
 'WindGustSpeed': '46',
 'WindDir9am': 'W',
 'WindDir3pm': 'WSW',
 'WindSpeed9am': '19',
 'WindSpeed3pm': '26',
 'Humidity9am': 38.0,
 'Humidity3pm': 30.0,
 'Pressure9am': 1207.6,
 'Pressure3pm': 1108.7,
 'Cloud9am': '2',
 'Cloud3pm': '2',
 'Temp9am': 21.0,
 'Temp3pm': 23.2,
 'RainToday': 'No'}]}
data_test = json.dumps(data['data'])
response = requests.post(uri, data=data_test, headers=headers)
print(response.text)

["No", "No"]


In [83]:
import json

data = df_rain[:5].drop(['RainTomorrow'], axis=1).to_json(orient='records')
service.run(data)

['No', 'No', 'No', 'No', 'No']

In [53]:
uri

'http://c6c55eb5-e54b-42f1-a896-2eb8671851a0.brazilsouth.azurecontainer.io/score'

In [60]:
import numpy as np
import pandas as pd
print(np.array(json.loads(data_test)))
m1.predict(pd.DataFrame.from_dict(json.loads(data_test))).tolist()

[{'Date': '2008-12-03', 'Location': 'Albury', 'MinTemp': 12.9, 'MaxTemp': 25.7, 'Rainfall': '0', 'Evaporation': 10, 'Sunshine': 2, 'WindGustDir': 'WSW', 'WindGustSpeed': '46', 'WindDir9am': 'W', 'WindDir3pm': 'WSW', 'WindSpeed9am': '19', 'WindSpeed3pm': '26', 'Humidity9am': 38.0, 'Humidity3pm': 30.0, 'Pressure9am': 1007.6, 'Pressure3pm': 1008.7, 'Cloud9am': 31, 'Cloud3pm': '2', 'Temp9am': 21.0, 'Temp3pm': 23.2, 'RainToday': 'No'}
 {'Date': '2008-12-03', 'Location': 'Albury', 'MinTemp': 22.9, 'MaxTemp': 45.7, 'Rainfall': '0', 'Evaporation': 20, 'Sunshine': 10, 'WindGustDir': 'WSW', 'WindGustSpeed': '46', 'WindDir9am': 'W', 'WindDir3pm': 'WSW', 'WindSpeed9am': '19', 'WindSpeed3pm': '26', 'Humidity9am': 38.0, 'Humidity3pm': 30.0, 'Pressure9am': 1207.6, 'Pressure3pm': 1108.7, 'Cloud9am': '2', 'Cloud3pm': '2', 'Temp9am': 21.0, 'Temp3pm': 23.2, 'RainToday': 'No'}]


['No', 'No']

In [68]:
uri

'http://97066473-954f-44d5-bbad-5a14f78fb753.brazilsouth.azurecontainer.io/score'

TODO: In the cell below, print the logs of the web service and delete the service

In [84]:
service.delete()

In [ ]:
#cpu_cluster.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
